While exploring AI agents, I came across two interesting libraries - Pydantic and Logfire. In this article, you will learn about Pydantic with code examples and understand what Pydantic brings to the table in the world of Data validation for Python developers.

Pydantic is a powerful Python library that uses type annotations to validate data structures. It's become an essential tool for many Python developers, especially those working on web applications, APIs, and data-intensive projects.

In [1]:
#  Install Pydantic
!pip install pydantic
!pip install pydantic_settings


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
Using cached pydantic_settings-2.7.1-py3-none-any.whl (29 kB)

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Base Model Definition
from pydantic import BaseModel
from typing import List, Optional


class User(BaseModel):
    username: str
    age: int
    email: str
    is_active: bool = True
    tags: List[str] = []
    profile_picture: Optional[str] = None

user = User(username="johndoe", age=30, email="john@example.com", tags=["python", "developer"])
print(user)

username='johndoe' age=30 email='john@example.com' is_active=True tags=['python', 'developer'] profile_picture=None


In [3]:
# Base Model Definition example - 2

from typing import List, Optional
from pydantic import BaseModel

class Book(BaseModel):
    title: str
    author: str
    publication_year: int
    isbn: str
    genres: List[str]
    description: Optional[str] = None


# Creating a valid book instance

book = Book(
    title="The Hitchhiker's Guide to the Galaxy",
    author="Douglas Adams",
    publication_year=1979,
    isbn="0-330-25864-8",
    genres=["Science Fiction", "Comedy"],
)


print(book)

title="The Hitchhiker's Guide to the Galaxy" author='Douglas Adams' publication_year=1979 isbn='0-330-25864-8' genres=['Science Fiction', 'Comedy'] description=None


In [3]:
# Nested Models

from pydantic import BaseModel
from typing import List


class Address(BaseModel):
	street: str
	city: str
	country: str
	postal_code: str


class Author(BaseModel):
	name: str
	age: int
	address: Address


class Book(BaseModel):
	title: str
	author: Author
	genres: List[str]


# Creating a book with nested author and address

book = Book(
	title="1984",
	author=Author(
		name="George Orwell",
		age=46,
		address=Address(
			street="50 Lawford Road",
			city="London",
			country="United Kingdom",
			postal_code="N1 5BJ"
		)
	),
	genres=["Dystopian", "Political Fiction"]
)

print(book)

title='1984' author=Author(name='George Orwell', age=46, address=Address(street='50 Lawford Road', city='London', country='United Kingdom', postal_code='N1 5BJ')) genres=['Dystopian', 'Political Fiction']


In [4]:
# Custom Validators 

import re
from pydantic import BaseModel, field_validator


class User(BaseModel):
	username: str
	email: str
	password: str

@field_validator('username')
def username_alphanumeric(cls, v):
	assert v.isalnum(), 'Username must be alphanumeric'
	return v

@field_validator('email')
def email_valid(cls, v):
	regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
	assert re.match(regex, v), 'Invalid email format'
	return v

@field_validator('password')
def password_strength(cls, v):
	assert len(v) >= 8, 'Password must be at least 8 characters'
	assert any(c.isupper() for c in v), 'Password must contain an uppercase letter'
	assert any(c.islower() for c in v), 'Password must contain a lowercase letter'
	assert any(c.isdigit() for c in v), 'Password must contain a digit'
	return v


# Try creating users

try:
	user1 = User(username="john_doe", email="john@example.com", password="StrongPass1")
	print("Valid user:", user1)

except ValueError as e:
	print("Validation error:", e)

try:
	user2 = User(username="alice!", email="invalid-email", password="weak")
    
except ValueError as e:
	print("Validation error:", e)

Valid user: username='john_doe' email='john@example.com' password='StrongPass1'


In [4]:
# Config and Field Constraints
from typing import List
from pydantic import BaseModel, Field


class Product(BaseModel):
	id: int = Field(..., gt=0)
	name: str = Field(..., min_length=3, max_length=50)
	price: float = Field(..., ge=0)
	tags: List[str] = Field(default_factory=list, max_length=5)


class Config:
	allow_mutation = False
	extra = "forbid"


# Creating a valid product

product = Product(id=1, name="Laptop", price=999.99, tags=["electronics", "computer"])
print(product)

# Attempting to create an invalid product

try:
	invalid_product = Product(
		id=0, name="TV", price=-100, tags=["a", "b", "c", "d", "e", "f"]
	)

except ValueError as e:
	print("Validation error:", e)

# Attempting to modify the product (which is not allowed due to allow_mutation=False)

try:
	product.price = 899.99
except AttributeError as e:
	print("Modification error:", e)

id=1 name='Laptop' price=999.99 tags=['electronics', 'computer']
Validation error: 4 validation errors for Product
id
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than
name
  String should have at least 3 characters [type=string_too_short, input_value='TV', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/string_too_short
price
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than_equal
tags
  List should have at most 5 items after validation, not 6 [type=too_long, input_value=['a', 'b', 'c', 'd', 'e', 'f'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/too_long


In [5]:
# Working with JSON 

from typing import List
from pydantic import BaseModel

class Comment(BaseModel):
    id: int
    text: str

class Post(BaseModel):
    id: int
    title: str
    content: str
    comments: List[Comment]


# JSON data
json_data = """
{
"id": 1,
"title": "Hello, Pydantic!",
"content": "This is a post about Pydantic.",
"comments": [
{"id": 1, "text": "Great post!"},
{"id": 2, "text": "Thanks for sharing."}
]
}
"""
# Parse JSON data into a Pydantic model
post = Post.model_validate_json(json_data)
print(post)

# Convert Pydantic model back to JSON
post_json = post.model_dump_json()
print(post_json)

id=1 title='Hello, Pydantic!' content='This is a post about Pydantic.' comments=[Comment(id=1, text='Great post!'), Comment(id=2, text='Thanks for sharing.')]
{"id":1,"title":"Hello, Pydantic!","content":"This is a post about Pydantic.","comments":[{"id":1,"text":"Great post!"},{"id":2,"text":"Thanks for sharing."}]}


In [8]:
# Settings Management
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    database_url: str
    api_key: str
    debug_mode: bool = False

settings = Settings()
print(settings)

ValidationError: 2 validation errors for Settings
database_url
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
api_key
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

In [10]:
# JSON Schema Generation
import json
print(json.dumps(User.model_json_schema()))

{"properties": {"username": {"title": "Username", "type": "string"}, "age": {"title": "Age", "type": "integer"}, "email": {"title": "Email", "type": "string"}, "is_active": {"default": true, "title": "Is Active", "type": "boolean"}, "tags": {"default": [], "items": {"type": "string"}, "title": "Tags", "type": "array"}, "profile_picture": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "title": "Profile Picture"}}, "required": ["username", "age", "email"], "title": "User", "type": "object"}
